# Team performance until the moment of the transfer

In [140]:
import pandas as pd
import numpy as np
import sys 
import csv
import unicodedata
import datetime as dt
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [141]:
d_tf = pd.read_csv('data_18.csv')

In [142]:
d_tf = d_tf.drop(columns=['Unnamed: 0'], axis=1)

In [143]:
# drop because we have no information about their performances

d_tf = d_tf[d_tf.team_from != 'Bastia']
d_tf = d_tf[d_tf.team_from != 'KAS Eupen']
d_tf = d_tf[d_tf.team_from != 'Cesena']
d_tf.shape

(767, 98)

# STATISTICS TEAM

T=total, S=successfull

In [144]:
def renameColumns(df):
    return df.rename(columns={
        'Shots / on target':'t_shots',
        'Unnamed: 9':'s_shots',
        'Unnamed: 10':'perc_succ_shots',
        'Passes / accurate':'t_passes', 
        'Unnamed: 12':'s_passes', 
        'Unnamed: 13':'perc_succ_passes', 
        'Possession, %':'perc_possession',
        'Losses / Low / Medium / High':'t_losses', 
        'Unnamed: 16':'low_losses', 
        'Unnamed: 17':'medium_losses',
        'Unnamed: 18':'high_losses', 
        'Recoveries / Low / Medium / High':'t_recoveries', 
        'Unnamed: 20':'low_recoveries',
        'Unnamed: 21':'medium_recoveries', 
        'Unnamed: 22':'high_recoveries', 
        'Challenges / won':'t_challenges', 
        'Unnamed: 24':'s_challenges',
        'Unnamed: 25':'perc_challenges_won'
    })

Function to construct 13 statistics for the teams

In [145]:
def goalsPerMatch(df):
    n_matches = df.shape[0]
    goal_per_match = round(sum(df.Goals)/n_matches,3)
    return goal_per_match

def totalGoals(df):
    tot_goals = sum(df.Goals)
    return tot_goals

def ExpectedGoals(df):
    n_matches = df.shape[0]
    xg = round(sum(df.xG)/n_matches,3)
    return xg

def percShotsOnTarget(df):
    perc_shots_target = round(sum(df.s_shots)/sum(df.t_shots)*100,2)
    return perc_shots_target

def shotsPerMatch(df):
    n_matches = df.shape[0]
    sum_col = df.sum(axis = 0, skipna = True) 
    t_shots = round(sum_col.t_shots/n_matches,2)
    return t_shots

def passesAccuracy(df):
    passes_acc = round(sum(df.s_passes)/sum(df.t_passes)*100,2) 
    return passes_acc

def passesPerMatch(df):
    n_matches = df.shape[0]
    sum_col = df.sum(axis = 0, skipna = True) 
    t_passes = round(sum_col.t_passes/n_matches,2)
    return t_passes

def meanPossession(df):
    n_matches = df.shape[0]
    perc_possession = round(sum(df.perc_possession)/n_matches,2)
    return perc_possession

def recoveriesPerMatch(df):
    n_matches = df.shape[0]
    sum_col = df.sum(axis = 0, skipna = True) 
    rec_tot = round(sum_col.t_recoveries/n_matches,2)
    return rec_tot

def propRecoveriesHigh(df):
    sum_col = df.sum(axis = 0, skipna = True) 
    rec_opposite = round(sum_col.high_recoveries/sum_col.t_recoveries,2)
    return rec_opposite

def lossesPerMatch(df):
    n_matches = df.shape[0]
    sum_col = df.sum(axis = 0, skipna = True) 
    los_per_match = round(sum_col.t_losses/n_matches,2)
    return los_per_match

def propLossesLow(df): # NB it is different from mean per match
    sum_col = df.sum(axis = 0, skipna = True) 
    los_own = round(sum_col.low_losses/sum_col.t_losses,2)
    return los_own   

def challengesPerMatch(df):
    n_matches = df.shape[0]
    sum_col = df.sum(axis = 0, skipna = True) 
    t_challenges = round(sum_col.t_challenges/n_matches,2)
    return t_challenges

def propSuccessfulChallenges(df):
    n_matches = df.shape[0]
    sum_col = df.sum(axis = 0, skipna = True) 
    perc_challenges_won = round(sum_col.perc_challenges_won/n_matches,2) #mean of the %
    return perc_challenges_won

In [146]:
#def keepBeforeDate(df,date):
#    df.Date = np.array(list(df.Date), dtype=np.datetime64)
#    df = df[df.Date <= date]
#    return df

def keepBeforeDate(df,date):
    date_compare = pd.to_datetime(date)
    date_format = '%Y-%m-%d'
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df['Date'] < date_compare]
    return df

def keepAfterDate(df,date):
    date_compare = pd.to_datetime(date)
    date_format = '%Y-%m-%d'
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df['Date'] >= date_compare]
    return df

In [147]:
# empty columns

d_tf['shots_per_match'] = np.nan
d_tf['perc_shots_target_team'] = np.nan
d_tf['xg_team'] = np.nan
d_tf['goals_per_match_team'] = np.nan
d_tf['passes_per_match'] = np.nan
d_tf['passes_accuracy'] = np.nan
d_tf['perc_possession_team'] = np.nan
d_tf['los_per_match_team'] = np.nan
d_tf['low_losses_team'] = np.nan
d_tf['rec_per_match_team'] = np.nan
d_tf['high_recoveries_team'] = np.nan
d_tf['challenges_per_match_team'] = np.nan
d_tf['perc_challenges_won_team'] = np.nan
d_tf['total_goals_opponents'] = np.nan
d_tf['total_goals'] = np.nan

In [148]:
# empty columns for 6 months before
d_tf['shots_per_match6Months'] = np.nan
d_tf['perc_shots_target_team6Months'] = np.nan
d_tf['xg_team6Months'] = np.nan
d_tf['goals_per_match_team6Months'] = np.nan
d_tf['passes_per_match6Months'] = np.nan
d_tf['passes_accuracy6Months'] = np.nan
d_tf['perc_possession_team6Months'] = np.nan
d_tf['los_per_match_team6Months'] = np.nan
d_tf['low_losses_team6Months'] = np.nan
d_tf['rec_per_match_team6Months'] = np.nan
d_tf['high_recoveries_team6Months'] = np.nan
d_tf['challenges_per_match_team6Months'] = np.nan
d_tf['perc_challenges_won_team6Months'] = np.nan
d_tf['total_goals_opponents6Months'] = np.nan
d_tf['total_goals6Months'] = np.nan

In [149]:
# empty months for 1 year before
d_tf['shots_per_match1Year'] = np.nan
d_tf['perc_shots_target_team1Year'] = np.nan
d_tf['xg_team1Year'] = np.nan
d_tf['goals_per_match_team1Year'] = np.nan
d_tf['passes_per_match1Year'] = np.nan
d_tf['passes_accuracy1Year'] = np.nan
d_tf['perc_possession_team1Year'] = np.nan
d_tf['los_per_match_team1Year'] = np.nan
d_tf['low_losses_team1Year'] = np.nan
d_tf['rec_per_match_team1Year'] = np.nan
d_tf['high_recoveries_team1Year'] = np.nan
d_tf['challenges_per_match_team1Year'] = np.nan
d_tf['perc_challenges_won_team1Year'] = np.nan
d_tf['total_goals_opponents1Year'] = np.nan
d_tf['total_goals1Year'] = np.nan

In [150]:
def remove_special (input_str):
    new = input_str.replace("'", "").replace("æ", "ae").replace("Ð", "D").replace("ð", "d").replace("Ø", "O").replace("ø", "o").replace("Æ", "AE").replace("œ", "oe").replace('ς','c').replace('đ','d')
    return new

In [151]:
d_tf.team_from = d_tf.team_from.replace('Darmstadt','Darmstadt 98').replace('Bournemouth','AFC Bournemouth').replace('Cardiff','Cardiff City').replace('Dusseldorf','Fortuna Dusseldorf').replace('Oostende','KV Oostende').replace('FC Ingolstadt','Ingolstadt')

In [152]:
def remove_accents(input_str):
    nkfd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nkfd_form if not unicodedata.combining(c)])

In [153]:
for i,row in d_tf.iterrows():
    date = row.date # takes the date of the transfer
    # date = np.array(date, dtype=np.datetime64) # trosformo in formato data
    team = row.team_from # takes the team relative to a player 

    # 1 open 4 files
    
    df = pd.read_excel("./Team_performance/Team Stats " + team + ".xlsx",encoding = "utf8")
    df1 = pd.read_excel("./Team_performance/Team Stats " + team + " (1).xlsx",encoding = "utf8")
    df2 = pd.read_excel("./Team_performance/Team Stats " + team + " (2).xlsx",encoding = "utf8")
    df3 = pd.read_excel("./Team_performance/Team Stats " + team + " (3).xlsx",encoding = "utf8")

    # 2 drop first lines with statistics on period different from the one we want
    
    df = df.drop(df.index[0:2]) # remove two white lines
    df1 = df1.drop(df1.index[0:2])
    df2 = df2.drop(df2.index[0:2])
    df3 = df3.drop(df3.index[0:2])

    df_tot = pd.concat([df,df1,df2,df3])
    df_tot['Team'] = df_tot.Team.apply(remove_accents).apply(remove_special)
    
    # 3 Select only the rows of the team taken into account
    df_opponents = df_tot[df_tot.Team != team] 
    df_tot = df_tot[df_tot.Team == team] 
    # select opponents to calculate the number of goals made by them

    # 4 Rename columns of the dataset
    df_tot = renameColumns(df_tot)
    df_opponents = renameColumns(df_opponents)
        
    # 5 Consider data only the date of the transfer
    df_tot = keepBeforeDate(df_tot, date)    
    df_opponents = keepBeforeDate(df_opponents, date) 
    
    # 6 STATISTICS
    
    total_goals_opponents = totalGoals(df_opponents)
    total_goals = totalGoals(df_tot)
    goals_per_match_team = goalsPerMatch(df_tot)
    xg_team = ExpectedGoals(df_tot)
    perc_shots_target_team = percShotsOnTarget(df_tot)
    shots_permatch = shotsPerMatch(df_tot)
    passes_accuracy = passesAccuracy(df_tot)
    passes_per_match = passesPerMatch(df_tot)
    perc_possession_team = meanPossession(df_tot)
    rec_per_match_team = recoveriesPerMatch(df_tot)
    high_recoveries_team = propRecoveriesHigh(df_tot)
    los_per_match_team = lossesPerMatch(df_tot)
    low_losses_team = propLossesLow(df_tot)
    challenges_per_match_team = challengesPerMatch(df_tot)
    perc_challenges_won_team = propSuccessfulChallenges(df_tot)

    d_tf.loc[i,'shots_per_match'] = shots_permatch
    d_tf.loc[i,'perc_shots_target_team'] = perc_shots_target_team
    d_tf.loc[i,'xg_team'] = xg_team
    d_tf.loc[i,'goals_per_match_team'] = goals_per_match_team
    d_tf.loc[i,'passes_per_match'] = passes_per_match
    d_tf.loc[i,'passes_accuracy'] = passes_accuracy
    d_tf.loc[i,'perc_possession_team'] = perc_possession_team
    d_tf.loc[i,'los_per_match_team'] = los_per_match_team
    d_tf.loc[i,'low_losses_team'] = low_losses_team
    d_tf.loc[i,'rec_per_match_team'] = rec_per_match_team
    d_tf.loc[i,'high_recoveries_team'] = high_recoveries_team
    d_tf.loc[i,'challenges_per_match_team'] = challenges_per_match_team
    d_tf.loc[i,'perc_challenges_won_team'] = perc_challenges_won_team
    d_tf.loc[i,'total_goals_opponents'] = total_goals_opponents
    d_tf.loc[i,'total_goals'] = total_goals
    

    # 7 compute 2 other dates corresponding to 6 month and 1 year before the transfer 
    date = pd.to_datetime(date)
    sixMonthsBefore = date + relativedelta(months= -6)
    oneYearBefore = date + relativedelta(months= -12)

    # 5b Consider data only before the date of the transfer but only in the year before the transfer
    df_tot = keepAfterDate(df_tot, oneYearBefore)    
    df_opponents = keepAfterDate(df_opponents, oneYearBefore) 
    
    # 6b STATISTICS
    
    total_goals_opponents = totalGoals(df_opponents)
    total_goals = totalGoals(df_tot)
    goals_per_match_team = goalsPerMatch(df_tot)
    xg_team = ExpectedGoals(df_tot)
    perc_shots_target_team = percShotsOnTarget(df_tot)
    shots_permatch = shotsPerMatch(df_tot)
    passes_accuracy = passesAccuracy(df_tot)
    passes_per_match = passesPerMatch(df_tot)
    perc_possession_team = meanPossession(df_tot)
    rec_per_match_team = recoveriesPerMatch(df_tot)
    high_recoveries_team = propRecoveriesHigh(df_tot)
    los_per_match_team = lossesPerMatch(df_tot)
    low_losses_team = propLossesLow(df_tot)
    challenges_per_match_team = challengesPerMatch(df_tot)
    perc_challenges_won_team = propSuccessfulChallenges(df_tot)

    d_tf.loc[i,'shots_per_match1Year'] = shots_permatch
    d_tf.loc[i,'perc_shots_target_team1Year'] = perc_shots_target_team
    d_tf.loc[i,'xg_team1Year'] = xg_team
    d_tf.loc[i,'goals_per_match_team1Year'] = goals_per_match_team
    d_tf.loc[i,'passes_per_match1Year'] = passes_per_match
    d_tf.loc[i,'passes_accuracy1Year'] = passes_accuracy
    d_tf.loc[i,'perc_possession_team1Year'] = perc_possession_team
    d_tf.loc[i,'los_per_match_team1Year'] = los_per_match_team
    d_tf.loc[i,'low_losses_team1Year'] = low_losses_team
    d_tf.loc[i,'rec_per_match_team1Year'] = rec_per_match_team
    d_tf.loc[i,'high_recoveries_team1Year'] = high_recoveries_team
    d_tf.loc[i,'challenges_per_match_team1Year'] = challenges_per_match_team
    d_tf.loc[i,'perc_challenges_won_team1Year'] = perc_challenges_won_team
    d_tf.loc[i,'total_goals_opponents1Year'] = total_goals_opponents
    d_tf.loc[i,'total_goals1Year'] = total_goals

    # 5c Consider data only the date of the transfer
    df_tot = keepAfterDate(df_tot, sixMonthsBefore)    
    df_opponents = keepAfterDate(df_opponents, sixMonthsBefore) 
    
    # 6c STATISTICS
    
    total_goals_opponents = totalGoals(df_opponents)
    total_goals = totalGoals(df_tot)
    goals_per_match_team = goalsPerMatch(df_tot)
    xg_team = ExpectedGoals(df_tot)
    perc_shots_target_team = percShotsOnTarget(df_tot)
    shots_permatch = shotsPerMatch(df_tot)
    passes_accuracy = passesAccuracy(df_tot)
    passes_per_match = passesPerMatch(df_tot)
    perc_possession_team = meanPossession(df_tot)
    rec_per_match_team = recoveriesPerMatch(df_tot)
    high_recoveries_team = propRecoveriesHigh(df_tot)
    los_per_match_team = lossesPerMatch(df_tot)
    low_losses_team = propLossesLow(df_tot)
    challenges_per_match_team = challengesPerMatch(df_tot)
    perc_challenges_won_team = propSuccessfulChallenges(df_tot)

    d_tf.loc[i,'shots_per_match6Months'] = shots_permatch
    d_tf.loc[i,'perc_shots_target_team6Months'] = perc_shots_target_team
    d_tf.loc[i,'xg_team6Months'] = xg_team
    d_tf.loc[i,'goals_per_match_team6Months'] = goals_per_match_team
    d_tf.loc[i,'passes_per_match6Months'] = passes_per_match
    d_tf.loc[i,'passes_accuracy6Months'] = passes_accuracy
    d_tf.loc[i,'perc_possession_team6Months'] = perc_possession_team
    d_tf.loc[i,'los_per_match_team6Months'] = los_per_match_team
    d_tf.loc[i,'low_losses_team6Months'] = low_losses_team
    d_tf.loc[i,'rec_per_match_team6Months'] = rec_per_match_team
    d_tf.loc[i,'high_recoveries_team6Months'] = high_recoveries_team
    d_tf.loc[i,'challenges_per_match_team6Months'] = challenges_per_match_team
    d_tf.loc[i,'perc_challenges_won_team6Months'] = perc_challenges_won_team
    d_tf.loc[i,'total_goals_opponents6Months'] = total_goals_opponents
    d_tf.loc[i,'total_goals6Months'] = total_goals

    
    #do only the first two players
    #if i==200:
    #    break

In [155]:
d_tf.to_csv('data_19_noGoalkeeper.csv', sep=',', header=True)